<style type="text/css" media="screen">
a:link { color:#FFFFFF; text-decoration: none; }
a:visited { color:#FFFFFF; text-decoration: none; }
a:hover { color:#FFFFFF; text-decoration: none; }
a:active { color:#FFFFFF; text-decoration: underline; }
</style>
# <A id="clone1" style="text-decoration:none;">HPE CSI Driver Data Fabric for Kubernetes</a>
![](imgs/hpedevlogo.svg)
<img src="imgs/mapr-ar21.gif" alt="csi logo" style="float: right;">
<img src="imgs/csilogo1.png" alt="csi logo" style="float: right;">
## How to deploy SQL Server 2019 with an Operator on Kubernetes Cluster

Emmanuel COLUSSI 
<br>
<br>
<br>
<br>

## Deploying SQL Server workload
### Create Clone Persistent Volume Claim (PVC)
</br>
<i class="fas fa-2x fa-info-circle" style="color:#FFAD33;"></i>&nbsp;&nbsp;<p>Cloning is the process of copying an online database onto another server. The copy is independent of the existing database and is preserved as a point-in-time snapshot.</p><p> You can use a cloned database for various purposes without putting a load on the production server or risking the integrity of production data. Some of these purposes include the following:</p>
<ul>
    <li>Performing analytical queries</li>
    <li>Load testing or integration testing of your apps</li>
   <li> Data extraction for populating data warehouses</li>
   <li> Running experiments on the data</li>
    <li>Migration ..... </li>
</ul>    



<p>We will demonstrate the ability to clone directly from a PVC as declared in the dataSource.It’s also possible to create forks of the SQL Server database to create even more sophisticated workflows.</p></br>

<p>It’s now possible to transform the data of the "test" deployment without disturbing the data of the "prod" deployment. This opens up the possibility to create advanced testing and development workflows that uses an exact representation of production data. Whether this dataset is a few bytes or a handful of terabytes, the operation will only take a few seconds to execute as the clones are not making any copies of the source data.</br>

We will create a Clone directly from an existing PVC without creating a snapshot. 

</p>
<p>The clone is a writable physical copy </p>
<img src="imgs/clone.gif" alt="csi logo" style="float: left;">



</br>
<p><i class="fas fa-2x fa-walking" style="color:#FFAD33;"></i>&nbsp;&nbsp;<b>STEP 1: </b>Now we can create a Volume Clone</p>
<p>We will create a Volume Clone <b><i>mapr-sqlserver-clone01-student4</i></b> which will be a Clone of the PVC <b><i>mapr-sqlserver-data01-student4</i></b>

<div class="alert" style="background-color:#FFAD33;color:#fff;" role="alert">
<p><i class="fas fa-file-alt"></i>&nbsp;&nbsp;Volume Clone file example : create-volumeclone-student4.yaml</p>
    </div>

<div class="alert" style="background-color:#FFAD33;color:#fff;" role="alert">
  <p class="mb-0"><i class="fas fa-sync-alt"></i>&nbsp;&nbsp;Run the following command in the next Cell with button <i class="fas fa-play"></i> : 

In [ ]:
ssh -t $JUPYTERHUB_USER@arvine "kubectl create -f  sql-mapr/create-volumeclone-student4.yaml -n mapr-$JUPYTERHUB_USER"

<div class="alert" style="background-color:#FFAD33;color:#fff;" role="alert">
  <p><i class="fas fa-eye"></i>&nbsp;&nbsp;show a Volume Clone is created : after about 1 minute the volume is cloned and will have the bound state .... <i class="fas fa-2x fa-smile"></i></p>
    <hr>
    <p class="mb-0"><i class="fas fa-sync-alt"></i>&nbsp;&nbsp;Run the following command in the next Cell with button <i class="fas fa-play"></i> :</p>
 
</div>

In [ ]:
ssh -t $JUPYTERHUB_USER@arvine "kubectl get pvc -n mapr-$JUPYTERHUB_USER |grep clone"

<i class="fas fa-2x fa-info-circle" style="color:#FFAD33;"></i>
> You can connect to the MapR management web interface and see the Clone volume created :
[HPE MapR Management](https://10.4.0.132:8443/app/mcs/#/app/volumes) </br>
<b>Username :</b> mapr</br>
<b>Password :</b>     

![](imgs/mapr-clone.png)

</br>
<i class="fas fa-2x fa-walking" style="color:#FFAD33;"></i>&nbsp;&nbsp;<b>STEP 2:</b> Now we can create a new SQL Server deployment using PVC : mapr-sqlserver-clone01-student4</br>

<div class="alert" style="background-color:#FFAD33;color:#fff;" role="alert">
  
  <p><i class="fas fa-file-export"></i>&nbsp;&nbsp;Deployement file example : deploy-mssql-s-clone.yaml</p>
 
</div>

<div class="alert" style="background-color:#FFAD33;color:#fff;" role="alert">
    <p class="mb-0"><i class="fas fa-sync-alt"></i>&nbsp;&nbsp;Run the command in the next Cells with button <i class="fas fa-play"></i> : </p>
</div> 

In [ ]:
ssh -t $JUPYTERHUB_USER@arvine "kubectl create -f sql-mapr/deploy-mssql-clone-$JUPYTERHUB_USER.yaml -n mapr-$JUPYTERHUB_USER"

The deployment and service are created. The SQL Server instance is in a container, connected to persistent storage.
<div class="alert" style="background-color:#FFAD33;color:#fff;" role="alert">
  
  <p><i class="fas fa-check-circle"></i>&nbsp;&nbsp;To view the status of the pod</p>
    <hr>
    <p class="mb-0"><i class="fas fa-sync-alt"></i>&nbsp;&nbsp;Run the command in the next Cell with button <i class="fas fa-play"></i> several times until the status is running</p>
 
</div>

In [ ]:
ssh -t $JUPYTERHUB_USER@arvine "kubectl get pods -n mapr-$JUPYTERHUB_USER"

The deployment and service are created. The SQL Server clone instance is in a container, connected to persistent storage.
<div class="alert" style="background-color:#FFAD33;color:#fff;" role="alert">
    <p class="mb-0"><i class="fas fa-sync-alt"></i>&nbsp;&nbsp;Run the commands in the next Cells with button <i class="fas fa-play"></i> : </p>
    <ul>
        <li> Get NodePort : TCP port binding SQL Server clone service</li>
        <li> Set varibale HOST_NAME : IP_NODE_KUBERNETES,TPC_PORT</li>
        <li> Check SQL Server Connexion</li>
    </ul>    
  <p><i class="fas fa-list-ol"></i>&nbsp;&nbsp;Run a query to view the top 10 items in the <b>Warehouse.StockItems tabler</b> , we have the same result as on the query executed on the source of the clone </p>   
 
</div> 

In [ ]:
export NODE_PORT=$(ssh $JUPYTERHUB_USER@arvine "kubectl get svc mssql-service-clone-$JUPYTERHUB_USER -n mapr-$JUPYTERHUB_USER -o jsonpath=\"{.spec.ports[0].nodePort}\" ") 

In [ ]:
export HOST_NAME="arvine,$NODE_PORT"

In [ ]:
ssh -t $JUPYTERHUB_USER@arvine "/opt/mssql-tools/bin/sqlcmd -U sa -P HPeinvent@ -S $HOST_NAME -Q \"SELECT TOP 10 StockItemID, StockItemName FROM WideWorldImporters.Warehouse.StockItems ORDER BY StockItemID\" "

</br></br>
<div class="alert" style="background-color:#FFAD33;color:#fff;width:100%" role="alert">
  <p style="font-size:20px;display: block;text-align:center;margin-left: auto;margin-right: auto;"><i class="fas fa-4x fa-smile"></i>&nbsp;&nbsp;Congratulations on completing the module&nbsp;&nbsp;<i class="fas fa-4x fa-praying-hands"></i></p>
</div>  

<br><br>

## <i class="fas fa-2x fa-map-marker-alt" style="color:#FFAD33;"></i>&nbsp;&nbsp;Next Steps

Create volumes Snapshot Persistent Volume Claim (PVC)


</br></br>
 <a href="HPEMAPR-SQL-snap.ipynb#snap" target="New" title="Next: Create volumes Snapshot Persistent Volume Claim (PVC)"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#FFAD33;color:#fff;position:relative;width:10%; height: 30px;float: right;"><b>Next</b></button></a>
 <a href="HPEMAPR-SQL-DB1.ipynb#DB1" target="New" title="Back: Use the persisted data"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#FFAD33;color:#fff;position:relative;width:10%; height: 30px;float: left;"><b>Back</b></button></a>

<p style="display: block;margin-left: auto;margin-right: auto;width:10%;" ><a href="#clone1" title="TOP"><i class="fas fa-2x fa-chevron-circle-up" style="color:#FFAD33;"></i></a></p>